In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset NA
df = pd.read_excel(r"C:\Users\sabri\NATeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,loss,FURIA,SPACESTATION,584,1,0,0,9,5,11.111111,3,5,0.000000
1,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,win,SPACESTATION,FURIA,1375,4,1,6,5,9,80.000000,5,3,0.666667
2,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,win,FURIA,G2 ESPORTS,1145,3,3,5,9,5,33.333333,0,1,1.000000
3,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,loss,G2 ESPORTS,FURIA,630,0,0,3,5,9,0.000000,1,0,0.333333
4,005c9d2c-6626-4b01-8b75-8f2cacbcf05a,NA I FaZe vs AXLE G1 2022-10-21.14.00,Mannfield (Night),2022-10-21,loss,TEAM AXLE,FAZE CLAN,792,1,0,4,7,9,14.285714,3,0,0.444444


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,loss,FURIA,SPACESTATION,584,1,0,0,9,5,11.111111,3,5,0.000000
1,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,win,SPACESTATION,FURIA,1375,4,1,6,5,9,80.000000,5,3,0.666667
2,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,win,FURIA,G2 ESPORTS,1145,3,3,5,9,5,33.333333,0,1,1.000000
3,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,loss,G2 ESPORTS,FURIA,630,0,0,3,5,9,0.000000,1,0,0.333333
4,005c9d2c-6626-4b01-8b75-8f2cacbcf05a,NA I FaZe vs AXLE G1 2022-10-21.14.00,Mannfield (Night),2022-10-21,loss,TEAM AXLE,FAZE CLAN,792,1,0,4,7,9,14.285714,3,0,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,ffb455ae-c3ef-40ca-a6b2-94b1980aa73d,NA G2 vs GGM1 G2 2022-10-21.15.44,Aquadome,2022-10-21,loss,GENGMOBIL1,G2 ESPORTS,811,1,1,3,3,7,33.333333,5,3,0.428571
2692,ffbf6720-636e-4e9f-9960-1a34a115da2e,NA F COL vs KOI G3 2023-02-18.14.30,DFH Stadium,2023-02-18,win,COMPLEXITY,KOI,2108,5,4,8,18,13,27.777778,5,5,0.615385
2693,ffbf6720-636e-4e9f-9960-1a34a115da2e,NA F COL vs KOI G3 2023-02-18.14.30,DFH Stadium,2023-02-18,loss,KOI,COMPLEXITY,1875,4,3,8,13,18,30.769231,5,5,0.444444
2694,ffc5ad44-c485-4816-b103-2e07bdd6b902,NA E SSG vs FAZE G4 2022-11-06.13.15,Utopia Coliseum (Dusk),2022-11-06,loss,SPACESTATION,FAZE CLAN,1167,1,1,8,8,13,12.500000,3,1,0.615385


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,loss,FURIA,SPACESTATION,584,1,0,0,9,5,11.111111,3,5,0.000000
1,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,win,SPACESTATION,FURIA,1375,3,1,6,5,9,80.000000,5,3,0.666667
2,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,win,FURIA,G2 ESPORTS,1145,0,3,5,9,5,33.333333,0,1,1.000000
3,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,loss,G2 ESPORTS,FURIA,630,0,0,3,5,9,0.000000,1,0,0.333333
4,005c9d2c-6626-4b01-8b75-8f2cacbcf05a,NA I FaZe vs AXLE G1 2022-10-21.14.00,Mannfield (Night),2022-10-21,loss,TEAM AXLE,FAZE CLAN,792,1,0,4,7,9,14.285714,3,0,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,ffb455ae-c3ef-40ca-a6b2-94b1980aa73d,NA G2 vs GGM1 G2 2022-10-21.15.44,Aquadome,2022-10-21,loss,GENGMOBIL1,G2 ESPORTS,811,0,1,3,3,7,33.333333,5,3,0.428571
2692,ffbf6720-636e-4e9f-9960-1a34a115da2e,NA F COL vs KOI G3 2023-02-18.14.30,DFH Stadium,2023-02-18,win,COMPLEXITY,KOI,2108,1,4,8,18,13,27.777778,5,5,0.615385
2693,ffbf6720-636e-4e9f-9960-1a34a115da2e,NA F COL vs KOI G3 2023-02-18.14.30,DFH Stadium,2023-02-18,loss,KOI,COMPLEXITY,1875,1,3,8,13,18,30.769231,5,5,0.444444
2694,ffc5ad44-c485-4816-b103-2e07bdd6b902,NA E SSG vs FAZE G4 2022-11-06.13.15,Utopia Coliseum (Dusk),2022-11-06,loss,SPACESTATION,FAZE CLAN,1167,0,1,8,8,13,12.500000,3,1,0.615385


In [5]:
#  Checando se há empates nos resultados
df_draws = df[df['result']=='draw']
df_draws

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded


In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,FURIA,SPACESTATION,584,1,0,0,9,5,11.111111,3,5,0.000000,1
1,000a9269-1f60-4417-962f-095196c2c61d,NA AB SSG vs FUR G5 2023-05-06.16.02,NaN,2023-05-06,SPACESTATION,FURIA,1375,3,1,6,5,9,80.000000,5,3,0.666667,0
2,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,FURIA,G2 ESPORTS,1145,0,3,5,9,5,33.333333,0,1,1.000000,0
3,000c55c5-1ebc-401e-a5e0-5881d1b791b6,NA A G2 vs FUR G4 2022-10-08.15.24,Utopia Coliseum (Dusk),2022-10-08,G2 ESPORTS,FURIA,630,0,0,3,5,9,0.000000,1,0,0.333333,1
4,005c9d2c-6626-4b01-8b75-8f2cacbcf05a,NA I FaZe vs AXLE G1 2022-10-21.14.00,Mannfield (Night),2022-10-21,TEAM AXLE,FAZE CLAN,792,1,0,4,7,9,14.285714,3,0,0.444444,1


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2418, 12) (278, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(278, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1813, 9), (605, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.88      0.89      0.89       306
           1       0.89      0.88      0.88       299

    accuracy                           0.88       605
   macro avg       0.88      0.88      0.88       605
weighted avg       0.88      0.88      0.88       605



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.
cv = cv = KFold(n_splits=10, shuffle=True, random_state=10)


In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=10,
                           n_estimators=300)


In [16]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(n_estimators = 300, min_samples_leaf = 10, learning_rate = 0.05)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       308
           1       0.87      0.89      0.88       297

    accuracy                           0.88       605
   macro avg       0.88      0.88      0.88       605
weighted avg       0.88      0.88      0.88       605
               precision    recall  f1-score   support

           0       0.85      0.94      0.89       139
           1       0.94      0.83      0.88       139

    accuracy                           0.89       278
   macro avg       0.89      0.89      0.89       278
weighted avg       0.89      0.89      0.89       278



In [17]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       901
           1       0.93      0.94      0.94       912

    accuracy                           0.94      1813
   macro avg       0.94      0.94      0.94      1813
weighted avg       0.94      0.94      0.94      1813



In [18]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.021118625885820714, 0.08783520155318937, 0.2725035966079116, 0.06721870121408743, 0.04610638305555466, 0.2496824463501127, 0.006877367237884604, 0.008856126193330045, 0.23980155190210886]
